# Using LLMs as text classifiers with an sklearn interface

<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/github/skorch-dev/skorch/blob/master/notebooks/llm-classifier-demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/skorch-dev/skorch/blob/master/notebooks/llm-classifier-demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

The notebook requires Hugging Face `transformers` and `datasets` as additional dependencies. If you have not already installed it, you can do so like this:

`python -m pip install transformers datasets`

In [1]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch', 'transformers', 'datasets'])
except ImportError:
    pass

## Imports

In [2]:
import datasets
import numpy as np
import pandas as pd
import transformers
import torch
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV

In [3]:
# let's reduce some of the noise from transformers and datasets logs
transformers.logging.set_verbosity_warning()
datasets.logging.set_verbosity_warning()

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## load data

Load sentiment dataset

In [5]:
imdb = datasets.load_dataset('imdb').shuffle(seed=0)

Found cached dataset imdb (/home/vinh/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/vinh/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-79aee49c9f40dc82.arrow
Loading cached shuffled indices for dataset at /home/vinh/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-5a09ddfc1bd0fbc8.arrow
Loading cached shuffled indices for dataset at /home/vinh/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-f131e6602007628b.arrow


Limit to 100 samples. Using zero/few shot learning mostly makes sense when there are very few labeled samples.

In [6]:
X = imdb['train'][:100]['text']
y = imdb['train'][:100]['label']

In [7]:
labels = np.array(['negative', 'positive'])[y]

## Zero-shot classification

In [8]:
from skorch.llm import ZeroShotClassifier

### "train" zero shot classifier

For this notebook, we use a small LLM, `flan-t5-small`.

In [9]:
clf = ZeroShotClassifier('google/flan-t5-small', device=device, use_caching=False)

In [10]:
%time clf.fit(X=None, y=['positive', 'negative'])

CPU times: user 3 s, sys: 569 ms, total: 3.57 s
Wall time: 4.12 s


ZeroShotClassifier(device='cuda:0', model_name='google/flan-t5-small', use_caching=False)

In general, fitting is fast because, basically, nothing happens. If the LLM is not cached locally, it will, however, be downloaded from Hugging Face, which may take some time.

### evaluate

In [11]:
%time y_proba = clf.predict_proba(X)

Token indices sequence length is longer than the specified maximum sequence length for this model (844 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 27.8 s, sys: 1.38 s, total: 29.2 s
Wall time: 7.68 s


In [12]:
log_loss(y, y_proba)

0.2870120002577984

In [13]:
y_pred = y_proba.argmax(1)

In [14]:
accuracy_score(y, y_pred)

0.86

In [15]:
clf.predict(["A masterpiece, instant classic, 5 stars out of 5"])

array(['positive'], dtype='<U8')

### grid search the prompt

In [16]:
prompt0 = """You are a text classification assistant.

The text to classify:

```
{text}
```

Choose the label among the following possibilities with the highest probability.
Only return the label, nothing more:

{labels}

Your response:
"""

In [17]:
prompt1 = """Your task is to classify text.

Choose the label among the following possibilities with the highest probability.
Only return the label, nothing more:

{labels}

The text to classify:

```
{text}
```

Your response:
"""

In [18]:
params = {'prompt': [prompt0, prompt1]}

In [19]:
search = GridSearchCV(clf, param_grid=params, cv=2, scoring=['accuracy', 'neg_log_loss'], refit=False)

In [20]:
%time search.fit(X, labels)

Token indices sequence length is longer than the specified maximum sequence length for this model (844 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (885 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (844 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (885 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 1min 58s, sys: 5.66 s, total: 2min 4s
Wall time: 35.2 s


GridSearchCV(cv=2,
             estimator=ZeroShotClassifier(device='cuda:0', model_name='google/flan-t5-small', use_caching=False),
             param_grid={'prompt': ['You are a text classification assistant.\n'
                                    '\n'
                                    'The text to classify:\n'
                                    '\n'
                                    '```\n'
                                    '{text}\n'
                                    '```\n'
                                    '\n'
                                    'Choose the label among the following '
                                    'possibilities with the highest '
                                    'probability.\n'
                                    'Only return the label, nothing more:\n'
                                    '\n'
                                    '{labels}\n'
                                    '\n'
                                    'Your response:\n',
                                    'Your task is to classify text.\n'
                                    '\n'
                                    'Choose the label among the following '
                                    'possibilities with the highest '
                                    'probability.\n'
                                    'Only return the label, nothing more:\n'
                                    '\n'
                                    '{labels}\n'
                                    '\n'
                                    'The text to classify:\n'
                                    '\n'
                                    '```\n'
                                    '{text}\n'
                                    '```\n'
                                    '\n'
                                    'Your response:\n']},
             refit=False, scoring=['accuracy', 'neg_log_loss'])

grid search results:

In [21]:
pd.DataFrame(search.cv_results_)[['mean_test_accuracy', 'mean_test_neg_log_loss', 'param_prompt', 'mean_score_time']]

,mean_test_accuracy,mean_test_neg_log_loss,param_prompt,mean_score_time
0,0.86,-0.287012,You are a text classification assistant.\n\nTh...,7.147494
1,0.93,-0.246949,Your task is to classify text.\n\nChoose the l...,7.287168


**Conclusion**: `prompt1` is performing better. Mean test accuracy of 93% and log loss of 0.25 are pretty good, given that we use zero shot and don't perform any fine-tuning.

## Few-shot classification

In [22]:
from skorch.llm import FewShotClassifier
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

### train few shot classifier

Instead of passing the model name to initialize the classifier, as in `clf = FewShotClassifier('google/flan-t5-small')`, it is also possible to pass the model and tokenizer explicitly. This is a good option if you need more control over them. In our case, it amounts to the same result.

In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-small').to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-small')

Use `max_samples` samples from the training data for few shot prompting.

In [24]:
clf = FewShotClassifier(
    model=model, tokenizer=tokenizer, max_samples=5, use_caching=False
)

In [25]:
%time clf.fit(X[:5], labels[:5])

CPU times: user 998 µs, sys: 52 µs, total: 1.05 ms
Wall time: 451 µs


FewShotClassifier(model='T5ForConditionalGeneration', tokenizer='T5TokenizerFast', use_caching=False)

Show how the prompt looks like:

In [26]:
print(clf.get_prompt(X[5]))

You are a text classification assistant.

Choose the label among the following possibilities with the highest probability.
Only return the label, nothing more:

['negative', 'positive']

Here are a few examples:

```
i went to see this movie with a bunch of friends one night. I didn't really hear much about it. So I wasn't expecting anything. But after I saw it, I really liked it. Nicolas Cage and the rest of the cast were very good. But I do have to say Giovanni Ribisi's acting performace did need a little perking up. But such a small flaw, it could be overrided. <br /><br />Gone In 60 Seconds is about a retired car thief who must boost 60 rare and exotic cars in one night to save his brother's life. The movie is in no way predictable. So the ending should be a suprise. Think it's just another, fast car driving movie? Well you are partially right. There is much more to it. Everyone should take a look at this movie.
```

Your response:
positive

```
I watch lots of scary movies (or at 

### evaluate

In [27]:
%time y_proba = clf.predict_proba(X)

Token indices sequence length is longer than the specified maximum sequence length for this model (1620 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 1min 5s, sys: 2.85 s, total: 1min 8s
Wall time: 17.1 s


In [28]:
log_loss(y, y_proba)

0.23648173200601527

In [29]:
y_pred = y_proba.argmax(1)

In [30]:
accuracy_score(y, y_pred)

0.91

In [31]:
clf.predict(["Even if paid $1000, I would not watch this movie again"])

array(['negative'], dtype='<U8')

### grid search best number of few shot samples

Note that grid search will split `X` and `y` for each run. Since the few shot samples are taken from X and y, those will thus be different for each split, which could have a big influence on the performance of the model. If you always want to have the same few shot samples in each split, you should craft your own prompt with those examples and then use it with `ZeroShotClassifier`. Just ensure that those prompts are not part of the validation/test data!

In [32]:
params = {'max_samples': [3, 5, 7]}

In [33]:
search = GridSearchCV(clf, param_grid=params, cv=2, scoring=['accuracy', 'neg_log_loss'], refit=False)

In [34]:
%time search.fit(X, labels)

CPU times: user 6min 36s, sys: 18 s, total: 6min 54s
Wall time: 1min 43s


GridSearchCV(cv=2,
             estimator=FewShotClassifier(model=T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_...
), tokenizer=T5TokenizerFast(name_or_path='google/flan-t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_id_53>', '<extra_id_54>', '<extra_id_55>', '<extra_id_56>', '<extra_id_57>', '<extra_id_58>', '<extra_id_59>', '<extra_id_60>', '<extra_id_61>', '<extra_id_62>', '<extra_id_63>', '<extra_id_64>', '<extra_id_65>', '<extra_id_66>', '<extra_id_67>', '<extra_id_68>', '<extra_id_69>', '<extra_id_70>', '<extra_id_71>', '<extra_id_72>', '<extra_id_73>', '<extra_id_74>', '<extra_id_75>', '<extra_id_76>', '<extra_id_77>', '<extra_id_78>', '<extra_id_79>', '<extra_id_80>', '<extra_id_81>', '<extra_id_82>', '<extra_id_83>', '<extra_id_84>', '<extra_id_85>', '<extra_id_86>', '<extra_id_87>', '<extra_id_88>', '<extra_id_89>', '<extra_id_90>', '<extra_id_91>', '<extra_id_92>', '<extra_id_93>', '<extra_id_94>', '<extra_id_95>', '<extra_id_96>', '<extra_id_97>', '<extra_id_98>', '<extra_id_99>']}, clean_up_tokenization_spaces=True), use_caching=False),
             param_grid={'max_samples': [3, 5, 7]}, refit=False,
             scoring=['accuracy', 'neg_log_loss'])

In [35]:
pd.DataFrame(search.cv_results_)[['mean_test_accuracy', 'mean_test_neg_log_loss', 'param_max_samples', 'mean_score_time']]

,mean_test_accuracy,mean_test_neg_log_loss,param_max_samples,mean_score_time
0,0.93,-0.213186,3,12.812054
1,0.91,-0.232684,5,15.329026
2,0.91,-0.232505,7,23.456210


**Conclusion**: No significant change in accuracy but medium improvement in log loss compared to zero shot. Having more samples doesn't help.

## Debugging

Working with LLMs can be difficult because it is hard to know for certain if the prompt works well and if the LLM is capable of classifying the input. For this reason, skorch provides a few options to help identify these issues.

### Returning unnormalized probabilities

By default, the model will normalize the probabilities to sum to 1. This is what is expected when calling `predict_proba`. However, this can hide underlying issues. The LLM can in theory predict any token from its vocabulary, there is no guarantee that it will choose one of the provided labels. skorch will force the LLM to use one of the labels, but we also track the probabilities assigned, or not assigned, to these labels.

To give an example, for a given input, it's possible that the LLM predicts a probability of 10% that the label is 'negative' and 70% that it is 'positive'. By default, we normalize the probability to be 1, i.e. we return 0.125 and 0.875. The problem is that we would return the same normalized probabilities even if the model predicts 1% and 7%. But if the model predicts such low probabilities, there is probably something wrong and we would like to know about it.

For this reason, we added the option to disable the normalization of probabilities. Let's check how well our zero-shot flan-t5 model is doing without normalization:

In [36]:
clf = ZeroShotClassifier('google/flan-t5-small', use_caching=False, probas_sum_to_1=False)

In [37]:
clf.fit(X=None, y=['positive', 'negative'])

ZeroShotClassifier(model_name='google/flan-t5-small', probas_sum_to_1=False, use_caching=False)

In [38]:
y_proba = clf.predict_proba(X[:3])

In [39]:
y_proba

array([[0.55589342, 0.43614349],
       [0.56059146, 0.43085057],
       [0.94313842, 0.04362511]])

In [40]:
y_proba.sum(1)

array([0.99203691, 0.99144202, 0.98676353])

As you can see, the probabilities returned by flan-t5 are quite high. Without normalization, they still sum up to ~99%, which is very good.

Now let's take a look at an LLM that doesn't work well for this task, GPT2:

In [41]:
# note that since GPT2 is a decoder-only language model, we don't need to set use_caching=False
clf = ZeroShotClassifier('gpt2', probas_sum_to_1=False)

In [42]:
clf.fit(X=None, y=['positive', 'negative'])

ZeroShotClassifier(model_name='gpt2', probas_sum_to_1=False)

In [43]:
y_proba = clf.predict_proba(X[:3])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [44]:
y_proba

array([[3.86091414e-13, 1.38600503e-12],
       [2.50074673e-13, 8.08236941e-13],
       [3.82718732e-13, 1.23716673e-12]])

As we can see, the probabilities are really low, but if we had normalized them, we might not have noticed:

In [45]:
# normalize probabilities to sum up to 1
y_proba / y_proba.sum(1, keepdims=True)

array([[0.21787269, 0.78212731],
       [0.23629588, 0.76370412],
       [0.23626284, 0.76373716]])

This means we should probably use a different LLM or tinker with the prompt until we get better results.

### Specific actions when probabilities are low

We provide more options to identify low probabilities in a more way that does not require manually inspecting the probabilities. For this, we provide two arguments for `ZeroShotClassifier` and `FewShotClassifier`.

The first argument is called `error_low_prob`. It should be one of the following strings: `'ignore'`, `'warn'`, `'raise'`, or `'return_none'`.

By default, it is `'ignore'`, which means that nothing happens, no matter how low the predicted proabilities. By setting it to `'warn'`, there will be a warning when the total probabilities of at least one predicted sample is too low. Use this option if you want to get the result but be alerted about possible problems.

By passing `error_low_prob='raise'`, an error will be raised as soon as a sample with low total probabilities is encountered. This is useful if you want inference to stop immediately, instead of waiting for all predictions to be made.

Finally, you can set `error_low_prob='return_none'`. In this case, nothing changes when calling `predict_proba`. When calling `predict`, however, the probabilities for the samples will be checked and if they're too low, the prediction will be replaced by `None`. This is useful if the predictions are generally good, but some examples are, for one reason or another, hard to predict.

The second parameter, which should be used in conjunction with `error_low_prob`, is called `threshold_low_prob`. This is simply a float between 0 and 1 that indicates what the probability is that should be considered "low". Note that this value is compared to the _sum of the probability for all labels_ of a given sample. So when setting `threshold_low_prob=0.1`, and the probability for 'negative' is 0.05, but the probability for 'positive' is 0.2, this would be fine because in total, their probabilities exceed 0.1.

Let's see how this works in practice by using the option to raise an error and setting the threshold to 0.5:

In [46]:
# note that since GPT2 is a decoder-only language model, we don't need to set use_caching=False
clf = ZeroShotClassifier('gpt2', error_low_prob='raise', threshold_low_prob=0.5)

In [47]:
clf.fit(X=None, y=['positive', 'negative'])

ZeroShotClassifier(error_low_prob='raise', model_name='gpt2', threshold_low_prob=0.5)

In [48]:
try:
    clf.predict_proba(X[:3])
except Exception as exc:
    print("There was an error:", exc)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


There was an error: The sum of all probabilities is 0.000, which is below the minimum threshold of 0.500


As you can see, we indeed got an error, alerting us immediately to potential issues.

## Testing MNLI

An existing method is to use natural language inference (NLI). Compare the results to https://huggingface.co/facebook/bart-large-mnli, which is the most used zero shot classifier on Hugging Face.

In [49]:
from transformers import pipeline

In [50]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli', device=device)

In [51]:
%time preds = classifier(imdb['train'][:100]['text'], ['negative', 'positive'])

CPU times: user 13 s, sys: 7.69 ms, total: 13 s
Wall time: 13 s


In [52]:
y_proba = np.vstack([p['scores'] if p['labels'] == ['negative', 'positive'] else p['scores'][::-1] for p in preds])

In [53]:
accuracy_score(y, y_proba.argmax(1))

0.84

In [54]:
log_loss(y, y_proba)

0.3443705626436628

**Conclusion**: This model is slower than the tested zero shot classifier, it is less flexible (we cannot adjust prompt or other parameters), and it performs worse.

## Testing a standard machine learning solution

Use a standard TFIDF + logistic regression benchmark.

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

In [56]:
tfidf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression()),
])

In [57]:
params = {'tfidf__max_features': [500, 1000], 'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]}

In [58]:
search = GridSearchCV(
    tfidf, param_grid=params, cv=2, scoring=['accuracy', 'neg_log_loss'], refit=False
)

In [59]:
%time search.fit(X, y)

CPU times: user 638 ms, sys: 95 µs, total: 638 ms
Wall time: 637 ms


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf', LogisticRegression())]),
             param_grid={'tfidf__max_features': [500, 1000],
                         'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             refit=False, scoring=['accuracy', 'neg_log_loss'])

The table is quite big, let's look at the top 5 best log losses:

In [60]:
cols = ['mean_test_accuracy', 'mean_test_neg_log_loss', 'param_tfidf__max_features', 'param_tfidf__ngram_range']
pd.DataFrame(search.cv_results_)[cols].sort_values('mean_test_neg_log_loss', ascending=False).head()

,mean_test_accuracy,mean_test_neg_log_loss,param_tfidf__max_features,param_tfidf__ngram_range
2,0.69,-0.662397,500,"(1, 3)"
5,0.71,-0.663959,1000,"(1, 3)"
1,0.68,-0.664004,500,"(1, 2)"
4,0.70,-0.664215,1000,"(1, 2)"
0,0.65,-0.664609,500,"(1, 1)"


**Conclusion**: This classical model is much faster, even if we include the training time, because it is much smaller than an LLM. However, it's scores are also much worse, given the small dataset. If speed is no concern, using an LLM classifier would thus be a good option for this task.